In [15]:
import sys
from pathlib import Path

project_root = Path().resolve().parent
sys.path.append(str(project_root))

import pandas as pd
from utils.insurance_pipeline import build_X_y


In [16]:
X, y, df_clean = build_X_y("../data/insurance.csv", include_bmi_smoker=True)

#### Q1. How much more do smokers cost on average?

In [3]:
df_clean.groupby('smoker')['charges'].agg(['count', 'mean', 'median']).round(2)

,count,mean,median
smoker,,,
no,1063,8440.66,7345.73
yes,274,32050.23,34456.35


#### Q2. How do costs vary by age group?

In [9]:
df_clean['age_group'] = pd.cut(
    df_clean['age'],
    bins=[17, 29, 39, 49, 59, 100],
    labels=['18–29','30–39','40–49','50–59','60+']
)

df_clean.groupby(
    'age_group',
    observed=True
)['charges'].agg(['count','mean','median']).round(2)



,count,mean,median
age_group,,,
18–29,416,9200.62,3220.37
30–39,257,11738.78,6082.40
40–49,279,14399.20,8606.22
50–59,271,16495.23,11729.68
60+,114,21248.02,14255.40


#### Q3. Do regions differ meaningfully?

In [5]:
df_clean.groupby('region')['charges'].agg(['count','mean','median']).round(2)

,count,mean,median
region,,,
northeast,324,13406.38,10057.65
northwest,324,12450.84,8976.98
southeast,364,14735.41,9294.13
southwest,325,12346.94,8798.59


#### Q4. Does BMI matter more for smokers?

In [7]:
df_clean.assign(
    bmi_smoker=df_clean['bmi'] * (df_clean['smoker']=='yes').astype(int)
).groupby('smoker')['charges'].agg(['mean','median']).round(2)


,mean,median
smoker,,
no,8440.66,7345.73
yes,32050.23,34456.35


#### Q5. Who are the highest-cost customers?

In [8]:
df_clean.sort_values('charges', ascending=False).head(10)

,age,sex,bmi,children,smoker,region,charges,smoker_flag,log_charges,age_group
543,54,female,47.410,0,yes,southeast,63770.42801,1,11.063045,50–59
1300,45,male,30.360,0,yes,southeast,62592.87309,1,11.044407,40–49
1230,52,male,34.485,3,yes,northwest,60021.39897,1,11.002456,50–59
577,31,female,38.095,1,yes,northeast,58571.07448,1,10.977996,30–39
819,33,female,35.530,0,yes,northwest,55135.40209,1,10.917547,30–39
1146,60,male,32.800,0,yes,southwest,52590.82939,1,10.870297,60+
34,28,male,36.400,1,yes,southwest,51194.55914,1,10.843389,18–29
1241,64,male,36.960,2,yes,southeast,49577.66240,1,10.811296,60+
1062,59,male,41.140,1,yes,southeast,48970.24760,1,10.798968,50–59
488,44,female,38.060,0,yes,southeast,48885.13561,1,10.797229,40–49


## SQL Ready Dataset

In [10]:
sql_df = df_clean.copy()

# keep engineered fields used later
sql_df['bmi_smoker'] = sql_df['bmi'] * (sql_df['smoker']=='yes').astype(int)
sql_df['log_charges'] = y.values

sql_df.head()


,age,sex,bmi,children,smoker,region,charges,smoker_flag,log_charges,age_group,bmi_smoker
0,19,female,27.900,0,yes,southwest,16884.92400,1,9.734176,18–29,27.9
1,18,male,33.770,1,no,southeast,1725.55230,0,7.453302,18–29,0.0
2,28,male,33.000,3,no,southeast,4449.46200,0,8.400538,18–29,0.0
3,33,male,22.705,0,no,northwest,21984.47061,0,9.998092,30–39,0.0
4,32,male,28.880,0,no,northwest,3866.85520,0,8.260197,30–39,0.0


In [11]:
sql_df.to_csv("../data/insurance_sql_ready.csv", index=False)

In [12]:
sql_df.isnull().sum()

age            0
sex            0
bmi            0
children       0
smoker         0
region         0
charges        0
smoker_flag    0
log_charges    0
age_group      0
bmi_smoker     0
dtype: int64

In [13]:
sql_df.shape

(1337, 11)

## Findings

- Smoking statusshows the largest cost defferential
- Median costs provide a more stable view than means due to skew
- Regional differences are present; however, their impact is minimal and not meaningful when evaluated against the influence of other variables.
- People with extreme costs are mostly older and they are all smokers.